In [1]:
from fastai.text import *
from fastai.datasets import *
from pathlib import Path
import pandas as pd
from fastai.metrics import *
from fastai.train import *
from fastai.vision import *
from fastai.imports import nn, torch
from sklearn import metrics
from fastai.callbacks import *

import sklearn.metrics
import datetime
torch.cuda.set_device(2)

import fastai

In [2]:
# only vary this
ner = True
max_vocab = 60000

exp_folder = str(max_vocab) + ('_ner' if ner else '')

EX_PA = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/' + exp_folder)
LM_DATA_PATH = Path('/mnt/data/group07/johannes/ynacc_proc/replicate/lmdata/')
exp_folder

'60000_ner'

In [3]:
# Language model data
if ner:
    data_train = pd.read_csv(LM_DATA_PATH/'train_proc_ner.csv')
    data_val = pd.read_csv(LM_DATA_PATH/'val_proc_ner.csv')
    data_train = data_train[['class', 'text_proc']]
    data_val = data_val[['class', 'text_proc']]
else:
    data_train = pd.read_csv(LM_DATA_PATH/'train_proc.csv', header=None, names=['text', 'text_proc'], index_col=0)
    data_val = pd.read_csv(LM_DATA_PATH/'val_proc.csv', header=None, names=['text', 'text_proc'], index_col=0)
    data_train = data_train[['text_proc']]
    data_val = data_val[['text_proc']]
    data_train = data_train.reset_index()
    data_val = data_val.reset_index()

In [4]:
data_train

,class,text_proc
0,1,It does help that these actors get paid huge s...
1,1,Read the xxperson comments
2,1,I rarely watch films with monsters in make-up....
3,1,Then you should not comment at all.
4,1,Why even bother reading the article?
5,1,"Well Fusion, do you prefer the real thing."
6,1,"xxordinal of all, it shouldn't be called the '..."
7,1,Chicago?
8,1,"Cheap Trick has been major influence on punk, ..."
9,1,Truer words have never written.


In [5]:
data_val

,class,text_proc
0,1,This would have been perfect if the B word was...
1,1,Well xxperson you sound very very frustrated b...
2,1,Tom: There are homicides in any country. Excep...
3,1,"It's not becuase of your age, it's because of ..."
4,1,Ummmm Suzi...even Trump supporters can spell '...
5,1,right LOL ...funny.. we do..everyday.. we than...
6,1,"their kids are gone, they've paid the ultimate..."
7,1,"their kids are gone, they've paid the ultimate..."
8,1,how do you know? not all xxnorp are alike.
9,1,International was the best truck.


In [6]:
data_lm = TextLMDataBunch.from_df(EX_PA, data_train, data_val, backwards=False, max_vocab=max_vocab, label_cols=0, text_cols=1)

In [7]:
data_lm.save()